In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F
from torch.utils.data import ConcatDataset, DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split

In [2]:
class CIFAR10Classifier(nn.Module):
    def __init__(self):
        super(CIFAR10Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(6272, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

dataset = ConcatDataset([train_dataset, test_dataset])
len_dataset = len(dataset)

model = CIFAR10Classifier()
state_dict = torch.load("/kaggle/input/private-model/model_state_dict.pth")
new_state_dict = {}
for key, value in state_dict.items():
    new_key = key.replace('_module.', '')  
    new_state_dict[new_key] = value

model.load_state_dict(new_state_dict)
model = model.to(device)
model.eval()
len_dataset

cuda


100%|██████████| 170498071/170498071 [00:08<00:00, 20971200.32it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


60000

In [4]:
def generate_synthetic_data(private_model, conf_min):
    private_model.eval()
    images, labels = [], []
    with torch.no_grad():
        for i in range(len_dataset):
            x,target = dataset[i]
            x = x.to(device)
            y = private_model(x.unsqueeze(0)) 
            y = torch.nn.functional.softmax(y, dim=1)
            prob = torch.max(y).item()
            
            if prob > conf_min:
                    images.append(x) , labels.append(target)
                    
    images = torch.stack(images)
    labels = torch.tensor(labels)
    
    return images, labels

class Synthetic(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

images, labels = generate_synthetic_data(model, 0.6)
synthetic_dataset = Synthetic(images, labels)
len(synthetic_dataset)

/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


12252

In [5]:
criterion = nn.CrossEntropyLoss()
synthetic_dataloader = DataLoader(synthetic_dataset,batch_size=64,shuffle=True)
correct = 0
running_loss = 0.0
for inputs,labels in synthetic_dataloader:
    with torch.no_grad():
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        predicted = np.argmax(outputs.cpu().numpy(), axis = 1)
        correct += np.sum(predicted == labels.cpu().numpy())
print(running_loss / len(synthetic_dataset))
correct / len(synthetic_dataset)

2.0832656792398763


0.5111002285341169

In [7]:
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

def test_model(model, data_loader):
    model.eval()
    all_outputs = []
    all_labels = []
    with torch.no_grad():
        correct = 0
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            predicted = np.argmax(outputs,axis=1)
            correct += np.sum(predicted == labels.cpu().numpy())
            outputs=-np.sort(-outputs , axis=1)
            all_outputs.append(torch.Tensor(outputs[:,:4]))
            all_labels.append(labels)
    print('accuracy : ' , correct / len(data_loader.dataset))
    return torch.cat(all_outputs), torch.cat(all_labels)

shadow_model_count = 10

shadow_train_outputs = []
shadow_train_labels = []
shadow_test_outputs = []
shadow_test_labels = []

for _ in range(shadow_model_count):
    train_size = int(0.5 * len(synthetic_dataset))
    val_size = len(synthetic_dataset) - train_size
    train_dataset_shadow , test_dataset_shadow = random_split(synthetic_dataset, [train_size,val_size])

    train_loader_shadow = DataLoader(train_dataset_shadow, batch_size = 8,shuffle = True)
    test_loader_shadow = DataLoader(test_dataset_shadow, batch_size = 64,shuffle = False)
    
    shadow_model = CIFAR10Classifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(shadow_model.parameters(), lr=0.001, weight_decay=1e-2)
    
    train_model(shadow_model, train_loader_shadow, criterion, optimizer , 3)
    
    train_outputs, train_labels = test_model(shadow_model, train_loader_shadow)
    shadow_train_outputs.append(train_outputs)
    shadow_train_labels.append(torch.ones_like(train_labels))  # Label 1 for seen data
    
    test_outputs, test_labels = test_model(shadow_model, test_loader_shadow)
    shadow_test_outputs.append(test_outputs)
    shadow_test_labels.append(torch.zeros_like(test_labels)) # Label 0 for un-seen data

all_train_outputs = torch.cat(shadow_train_outputs)
all_train_labels = torch.cat(shadow_train_labels)
all_test_outputs = torch.cat(shadow_test_outputs)
all_test_labels = torch.cat(shadow_test_labels)

attack_inputs = torch.cat((all_train_outputs, all_test_outputs))
attack_labels = torch.cat((all_train_labels, all_test_labels))

attack_features = attack_inputs.cpu().view(attack_inputs.size(0), -1).numpy()
attack_labels = attack_labels.cpu().numpy()

Epoch 1, Loss: 1.9183851303070705
Epoch 2, Loss: 1.7227359379084863
Epoch 3, Loss: 1.6377929649838578
accuracy :  0.5586026771139406
accuracy :  0.5466862553052563
Epoch 1, Loss: 1.9089541108427721
Epoch 2, Loss: 1.764962779946489
Epoch 3, Loss: 1.6607120577100052
accuracy :  0.5527260855370552
accuracy :  0.5471759712699967
Epoch 1, Loss: 1.9188995442253491
Epoch 2, Loss: 1.7382448399658303
Epoch 3, Loss: 1.6595583544854395
accuracy :  0.5287300032647732
accuracy :  0.5210577864838394
Epoch 1, Loss: 1.9342392897792646
Epoch 2, Loss: 1.7187658445642138
Epoch 3, Loss: 1.6405471410047914
accuracy :  0.5439111981717271
accuracy :  0.5156709108716944
Epoch 1, Loss: 1.8841518286625336
Epoch 2, Loss: 1.6841828297852848
Epoch 3, Loss: 1.5960128462968233
accuracy :  0.5463597779954293
accuracy :  0.5292197192295135
Epoch 1, Loss: 1.9148964681139815
Epoch 2, Loss: 1.7215863414128207
Epoch 3, Loss: 1.638641298984423
accuracy :  0.49918380672543256
accuracy :  0.4763303950375449
Epoch 1, Loss: 1.

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(attack_features, attack_labels, test_size=0.2)

attack_model = RandomForestClassifier(max_depth=6)
attack_model.fit(X_train, y_train)

attack_predictions = attack_model.predict(X_train)
attack_accuracy = accuracy_score(y_train, attack_predictions)
print(f'Attack Model Train Accuracy: {attack_accuracy * 100:.2f}%')

attack_predictions = attack_model.predict(X_test)
attack_accuracy = accuracy_score(y_test, attack_predictions)
print(f'Attack Model Test Accuracy: {attack_accuracy * 100:.2f}%')

Attack Model Train Accuracy: 54.27%
Attack Model Test Accuracy: 50.05%


In [10]:
from torch.utils.data import DataLoader, TensorDataset

X_train, X_test, y_train, y_test = train_test_split(attack_features, attack_labels, test_size=0.2)

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(4, 16)  
        self.fc2 = nn.Linear(16, 8) 
        self.fc3 = nn.Linear(8, 1) 
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x)) 
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

train_attack_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train).view(-1,1).to(torch.float32))
train_attack_loader = DataLoader(train_attack_dataset, batch_size=16, shuffle=True)

test_attack_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test).view(-1,1).to(torch.float32))
test_attack_loader = DataLoader(test_attack_dataset, batch_size=16, shuffle=False)

attacker = MLP().to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(attacker.parameters(), lr=0.01) 

def eval_attacker(attacker, loader, text):
    correct = 0
    with torch.no_grad():
        for batch_data, batch_targets in loader:
            batch_data, batch_targets = batch_data.to(device), batch_targets.to(device)
            outputs = attacker(batch_data)
            predicted = np.round(outputs.cpu().numpy())
            correct += np.sum(predicted == batch_targets.cpu().numpy())
    print(f'accuracy {text} : {correct / len(loader.dataset)} ')

num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0
    for batch_data, batch_targets in train_attack_loader:
        batch_data, batch_targets = batch_data.to(device), batch_targets.to(device)
        optimizer.zero_grad()
        outputs = attacker(batch_data)
        loss = criterion(outputs, batch_targets)
        running_loss += loss.item() * batch_data.size(0)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_attack_loader.dataset):.4f}')
    eval_attacker(attacker, train_attack_loader, 'train')
    eval_attacker(attacker, test_attack_loader, 'test')
torch.save(attacker.state_dict(), 'attacker.pth')

Epoch [1/5], Loss: 0.6933
accuracy train : 0.5008059908586353 
accuracy test : 0.5083251714005876 
Epoch [2/5], Loss: 0.6932
accuracy train : 0.5023771629121776 
accuracy test : 0.4953885079986941 
Epoch [3/5], Loss: 0.6932
accuracy train : 0.501540564805746 
accuracy test : 0.493837740777016 
Epoch [4/5], Loss: 0.6932
accuracy train : 0.5028974861247143 
accuracy test : 0.4951436500163239 
Epoch [5/5], Loss: 0.6931
accuracy train : 0.5052950538687562 
accuracy test : 0.5013059092393078 


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import Subset, DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, precision_score, recall_score ,f1_score
from sklearn.linear_model import LogisticRegression

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CIFAR10Classifier()
state_dict = torch.load("/kaggle/input/private-model/model_state_dict.pth", map_location=device)
new_state_dict = {key.replace('_module.', ''): value for key, value in state_dict.items()}
model.load_state_dict(new_state_dict)
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

DATA_ROOT = '../cifar10'
BATCH_SIZE = 1024

# Load the indices from list.txt
indices_file = '/kaggle/input/list-txt/list.txt' ############
with open(indices_file, 'r') as f:
    indices = [int(line.strip()) for line in f]

full_train_dataset = CIFAR10(root=DATA_ROOT, train=True, download=True, transform=transform)
test_dataset = CIFAR10(root=DATA_ROOT, train=False, download=True, transform=transform)

train_indices_set = set(indices)
all_indices = set(range(len(full_train_dataset)))
other_indices = list(all_indices - train_indices_set)

train_dataset = Subset(full_train_dataset, indices[:len(indices)//2])  ###########
other_dataset = Subset(full_train_dataset, other_indices)
print(len(train_dataset))
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
other_loader = DataLoader(other_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Create labels
train_labels = torch.ones(len(train_dataset)).to(device)
other_labels = torch.zeros(len(other_dataset)).to(device)
test_labels = torch.zeros(len(test_dataset)).to(device)
####################################
#if you have an attacker model for each class, modify the above code.
####################################

def extract_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for data in dataloader:
            inputs, _ = data
            inputs = inputs.to(device)
            outputs = F.softmax(model(inputs),dim=1).cpu().numpy()
            outputs=-np.sort(-outputs , axis=1)
            features.append(torch.Tensor(outputs[:,:4]))
    return torch.cat(features).to(device)

train_features = extract_features(model, train_loader)
other_features = extract_features(model, other_loader)
test_features = extract_features(model, test_loader)


combined_features = torch.cat((train_features, other_features, test_features))
combined_labels = torch.cat((train_labels, other_labels, test_labels))


new_dataset = TensorDataset(combined_features, combined_labels)
new_loader = DataLoader(new_dataset, batch_size=BATCH_SIZE, shuffle=True)

#load your attacker model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(4, 16)  
        self.fc2 = nn.Linear(16, 8) 
        self.fc3 = nn.Linear(8, 1) 
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x)) 
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
attacker = MLP().to(device)
attacker.load_state_dict(torch.load('/kaggle/working/attacker.pth'))
#############################################

# Calculate training accuracy, confusion matrix, precision, and recall
attacker.eval()
all_labels = []
all_predicted = []
correct = 0
total = 0

with torch.no_grad():
    for features, labels in new_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = attacker(features).squeeze()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predicted.extend(predicted.cpu().numpy())

accuracy = correct / total
print(f'Training Accuracy: {accuracy:.4f}')

cm = confusion_matrix(all_labels, all_predicted)
precision = precision_score(all_labels, all_predicted)
recall = recall_score(all_labels, all_predicted)
f1 = f1_score(all_labels, all_predicted)

print(f'Confusion Matrix:\n{cm}')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Files already downloaded and verified
Files already downloaded and verified
20000


/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Training Accuracy: 0.6696
Confusion Matrix:
[[10316  9684]
 [ 3532 16468]]
Precision: 0.6297
Recall: 0.8234
F1 Score: 0.7136
